In [ ]:


import os
from dotenv import load_dotenv
from openai import OpenAI
import opik
from opik import Opik, track
from opik.opik_context import update_current_trace

# ==========================================
# 1. 환경 변수 로드 및 SDK 매핑 (대화창 방지 핵심)
# ==========================================
load_dotenv()

# Opik SDK가 'opik.configure()' 없이도 설정을 인식하도록 환경 변수 강제 매핑
# .env에 정의된 값을 Opik 내부 시스템 변수명으로 변환합니다.
os.environ["OPIK_URL_OVERRIDE"] = os.getenv("OPIK_BASE_URL", "http://localhost:5173/api")
os.environ["OPIK_WORKSPACE"] = os.getenv("OPIK_WORKSPACE", "seocho-kgbuild")
os.environ["OPIK_PROJECT_NAME"] = os.getenv("OPIK_PROJECT_NAME", "kgbuild")

print("✅ 환경 변수 매핑 완료")

# ==========================================
# 2. 클라이언트 인스턴스 생성 (Singletons)
# ==========================================
try:
    # opik.configure()를 생략하고 바로 Opik()을 호출하여 조용한 연결 수행
    OPIK_CLIENT = Opik()
    
    # OpenAI 클라이언트 초기화
    OPENAI_CLIENT = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    print(f"✅ Opik 연결 성공: {OPIK_CLIENT.get_project_url()}")
    print(f"✅ OpenAI 클라이언트 준비 완료")
    
except Exception as e:
    print(f"❌ 초기화 중 오류 발생: {e}")

# ==========================================
# 2. 프롬프트 관리 (Registration & Versioning)
# ==========================================
def setup_prompts():
    """Opik 라이브러리에 사용할 프롬프트들을 등록하거나 업데이트합니다."""
    
    # 추출(Extraction) 프롬프트 - ChatPrompt 형식 (버전 관리 용이)
    OPIK_CLIENT.create_chat_prompt(
        name="fibo-extraction-chat",
        messages=[
            {"role": "system", "content": "당신은 FIBO 온톨로지 전문가입니다. 반드시 UpperCamelCase 형식을 유지하고 ISO 704 정의를 포함하세요."},
            {"role": "user", "content": "텍스트를 분석하여 JSON으로 엔티티를 추출하세요: {{text}}"}
        ],
        metadata={"domain": "finance", "ontology": "FIBO"}
    )
    
    # 연결(Linking) 프롬프트 - 일반 TextPrompt 형식
    OPIK_CLIENT.create_prompt(
        name="fibo-linking",
        prompt="""추출된 엔티티들을 FIBO 표준 IRI와 연결하세요. 
대상 엔티티: {{entities}}
결과를 RDF 트리플과 Neo4j Cypher 구조로 반환하세요.""",
        metadata={"type": "linking"}
    )
    print("✅ 프롬프트 라이브러리 세팅 완료")

# ==========================================
# 3. 코어 에이전트 파이프라인 (Core Logic)
# ==========================================

@track(name="step_extraction")
def step_extraction(raw_text, commit=None):
    """1단계: 텍스트에서 FIBO 기반 엔티티 추출"""
    # 특정 커밋 버전이 지정되지 않으면 최신 버전을 가져옵니다.
    prompt_template = OPIK_CLIENT.get_chat_prompt(name="fibo-extraction-chat", commit=commit)
    messages = prompt_template.format(variables={"text": raw_text})
    
    response = OPENAI_CLIENT.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        response_format={"type": "json_object"}
    )
    return response.choices[0].message.content, prompt_template

@track(name="step_linking")
def step_linking(entities_json):
    """2단계: 추출된 엔티티를 FIBO IRI 및 Graph 쿼리로 연결"""
    prompt_template = OPIK_CLIENT.get_prompt(name="fibo-linking")
    formatted_prompt = prompt_template.format(entities=entities_json)
    
    response = OPENAI_CLIENT.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": formatted_prompt}]
    )
    return response.choices[0].message.content, prompt_template

@track(name="fibo-main-pipeline")
def run_fibo_pipeline(input_text, extraction_commit=None):
    """추출과 연결을 수행하는 전체 RAG 파이프라인"""
    
    # Step 1: Extraction
    entities, extract_p_obj = step_extraction(input_text, commit=extraction_commit)
    
    # Step 2: Linking
    final_result, link_p_obj = step_linking(entities)
    
    # 트레이스 메타데이터 업데이트 (Prompts Management 연동)
    update_current_trace(
        prompts=[extract_p_obj, link_p_obj],
        metadata={
            "input_chars": len(input_text),
            "extraction_version": extract_p_obj.commit
        }
    )
    
    return final_result

# ==========================================
# 4. 실행 및 테스트
# ==========================================
if __name__ == "__main__":
    # 1. 초기 1회 프롬프트 등록 (이미 있다면 업데이트)
    setup_prompts()
    
    # 2. 테스트 데이터 실행
    sample_text = "Goldman Sachs issued a commercial paper in New York."
    
    # 특정 버전을 고정해서 실행하고 싶다면 commit hash를 전달
    # result = run_fibo_pipeline(sample_text, extraction_commit="your_commit_hash")
    result = run_fibo_pipeline(sample_text)
    
    print("\n🚀 파이프라인 실행 결과:")
    print(result)

In [ ]:
@opik.track
def evaluation_task(dataset_item):
    # dataset_item['input']['text']를 에이전트에게 전달
    raw_text = dataset_item["input"]["text"]
    
    # 우리가 만든 파이프라인 실행 (추출 -> 연결)
    generated_answer = run_fibo_pipeline(raw_text) 
    
    # 평가를 위해 생성된 답변과 검색된 컨텍스트를 반환
    return {
        "output": generated_answer,
        "context": [dataset_item["metadata"]["references"]] # Retrieval 품질 측정을 위함
    }

from opik.evaluation import evaluate
from opik.evaluation.metrics import AnswerRelevance, Hallucination

# 측정 지표 초기화
metrics = [
    AnswerRelevance(),
    Hallucination()
]

# 실험 실행
evaluation_results = evaluate(
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=metrics,
    experiment_name="fibo-agent-v1-test",
    nb_samples=10
)